In [1]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm 
from PIL import Image
from io import BytesIO
import re

In [2]:
train_data = pd.read_csv('/share/jproject/fg538/r-006-gpu-4/data/train.csv')
temp = train_data.groupby(['landmark_id']).size().reset_index(name='counts')
temp.sort_values('counts', ascending=False, inplace=True)

In [3]:
# Getting the top n landmark_ids with most samples
# Top n classes
temp_2 = np.array(temp.tail(-100)['landmark_id'])

In [4]:
# train data with only top n classes
# bacth 2: 450-1800
train_data_sub = train_data.loc[train_data['landmark_id'].isin(temp_2[:450])]
#train_data_sub.head()

In [5]:
TARGET_SIZE = 299

def overwrite_urls(df):
    def reso_overwrite(url_tail, reso=TARGET_SIZE):
        pattern = 's[0-9]+'
        search_result = re.match(pattern, url_tail)
        if search_result is None:
            return url_tail
        else:
            return 's{}'.format(reso)

    def join_url(parsed_url, s_reso):
        parsed_url[-2] = s_reso
        return '/'.join(parsed_url)

    parsed_url = df.url.apply(lambda x: x.split('/'))
    train_url_tail = parsed_url.apply(lambda x: x[-2])
    resos = train_url_tail.apply(lambda x: reso_overwrite(x, reso=TARGET_SIZE))

    overwritten_df = pd.concat([parsed_url, resos], axis=1)
    overwritten_df.columns = ['url', 's_reso']
    df['url'] = overwritten_df.apply(lambda x: join_url(x['url'], x['s_reso']), axis=1)
    return df

In [6]:
train_data_sub2 = overwrite_urls(train_data_sub)

/opt/Anaconda3-5.0.1/envs/e533/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
for i in tqdm(train_data_sub2.index):
    URL = train_data_sub2['url'][i]
    class_id = str(train_data_sub2['landmark_id'][i])
    image_name = train_data_sub2['id'][i]
    filename = '/share/jproject/fg538/r-006-gpu-4/data/train2/' + class_id + '/' + image_name + '.jpg'

    # skip download if it already exists
    if os.path.exists(filename):
        continue

    try:
        with urllib.request.urlopen(URL) as url:
            os.makedirs(os.path.dirname(filename), exist_ok=True)
            image = url.read()
    except:
        #tqdm.write('Could not download image: {}'.format(image_name)) # print() interferes with progress bar
        pass
        
    try:
        pil_image = Image.open(BytesIO(image))
    except:
        #tqdm.write('Failed to parse image: {}'.format(image_name))
        pass
        
    try:
        pil_image_rgb = pil_image.convert('RGB')
    except:
        #tqdm.write('Failed to convert image {} to RGB'.format(image_name))
        pass

    try:
        pil_image_resize = pil_image_rgb.resize((299, 299))
    except:
        pass
    
    try:
        pil_image_resize.save(filename, format='JPEG', quality=90)
    except:
        #tqdm.write('Failed to save image: {}'.format(filename))
        pass

 99%|█████████▉| 275194/276593 [13:36<00:04, 337.08it/s]

In [ ]:
5